In [164]:
from google.cloud import bigquery
import pandas as pd
import os

os.environ['GOOGLE_APPLICATION+CREDENTIALS'] = '/Users/nrabbani/.config/gcloud/application_default_credentials.json'
os.environ['GCLOUD_PROJECT'] = 'mining-clinical-decisions'
%load_ext google.cloud.bigquery

client=bigquery.Client()

The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery


/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/google/auth/_default.py:70: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/google/auth/_default.py:70: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For mor

In [165]:
query = '''
WITH lr_inpt as (
    SELECT * FROM `mining-clinical-decisions.lpch_core.lab_result`
    WHERE ordering_mode='Inpatient'
    Limit 10
)
SELECT *
FROM (
    SELECT
        ANY_VALUE(lr_inpt) lr_inpt,
        ARRAY_AGG(adt order by adt.event_time_jittered DESC LIMIT 1) adt
        FROM lr_inpt
        LEFT JOIN `mining-clinical-decisions.lpch_core.adt` adt
        USING (pat_enc_csn_id_coded)
        WHERE lr_inpt.order_time >= adt.EVENT_TIME_JITTERED
        GROUP BY TO_JSON_STRING(lr_inpt)
)
'''

query_job = client.query(query)
df = query_job.to_dataframe()

In [172]:
df

,lr_inpt,adt
0,"{'anon_id': 'JC799980', 'pat_enc_csn_id_coded'...","[{'ANON_ID': 'JC799980', 'PAT_ENC_CSN_ID_CODED..."
1,"{'anon_id': 'JC658803', 'pat_enc_csn_id_coded'...","[{'ANON_ID': 'JC658803', 'PAT_ENC_CSN_ID_CODED..."
2,"{'anon_id': 'JC618687', 'pat_enc_csn_id_coded'...","[{'ANON_ID': 'JC618687', 'PAT_ENC_CSN_ID_CODED..."
3,"{'anon_id': 'JC696156', 'pat_enc_csn_id_coded'...","[{'ANON_ID': 'JC696156', 'PAT_ENC_CSN_ID_CODED..."
4,"{'anon_id': 'JC711973', 'pat_enc_csn_id_coded'...","[{'ANON_ID': 'JC711973', 'PAT_ENC_CSN_ID_CODED..."
5,"{'anon_id': 'JC1146621', 'pat_enc_csn_id_coded...","[{'ANON_ID': 'JC1146621', 'PAT_ENC_CSN_ID_CODE..."


In [226]:
%%bigquery
SELECT pat_lv_of_care, COUNT(pat_lv_of_care) as cnt
FROM njr_labutil.lradt
GROUP BY pat_lv_of_care
ORDER BY cnt desc

,pat_lv_of_care,cnt
0,Critical Care,8495061
1,Acute Care,6762568
2,,3169406
3,Intermediate Intensive Care,542719
4,Well Infant Care,452042
5,Recovery/Post Anesthesia Care,27211


In [223]:
%%bigquery
SELECT pat_class, COUNT(pat_class) as cnt
FROM (
    SELECT * FROM njr_labutil.lradt
)
GROUP BY pat_class
ORDER BY cnt desc

,pat_class,cnt
0,Inpatient,12184370
1,Surgery Admit,2691918
2,Newborn,2597579
3,Outpatient,1486499
4,Hospital Outpatient Surgery,425804
5,Observation,60454
6,,1041
7,Radiation Oncology Series,914
8,Nuclear Med Series,423
9,OB Series,5


In [230]:
pd.set_option('display.max_rows', None)

In [231]:
%%bigquery
SELECT pat_service, COUNT(pat_service) as cnt
FROM (
    SELECT * FROM njr_labutil.lradt
)
GROUP BY pat_service
ORDER BY cnt desc



,pat_service,cnt
0,Neonatology,3252738
1,Cardiology,2670793
2,Intensive Care,2308405
3,Obstetrics,1875714
4,Hematology/Oncology,1847498
5,Cardiovascular Surgery,1333710
6,Stem Cell Transplant,869208
7,,798017
8,General Pediatrics,680400
9,Gastroenterology,560896


In [225]:
%%bigquery
SELECT accomodation, COUNT(accomodation) as cnt
FROM (
    SELECT * FROM njr_labutil.lradt
)
GROUP BY accomodation
ORDER BY cnt desc

,accomodation,cnt
0,,2216723
1,PEDS MED/SURG Level 2,1346219
2,Critical Care Level 3,1341010
3,Labor & Delivery,1289076
4,NICU Level 3 (174),1274545
5,CV Critical Care Level 3,1153808
6,Oncology Level 3,1089229
7,CV Critical Care Level 2,908903
8,PEDS MED/SURG Level 3,837399
9,SCT,774643


In [216]:
%%bigquery
SELECT pat_class, COUNT(pat_class) as cnt
FROM (
    SELECT * FROM njr_labutil.lradt
    WHERE pat_lv_of_care = ''
)
GROUP BY pat_class
ORDER BY cnt desc

,pat_class,cnt
0,Outpatient,1477223
1,Surgery Admit,687671
2,Inpatient,476340
3,Hospital Outpatient Surgery,413284
4,Newborn,82455
5,Observation,30050
6,,1041
7,Radiation Oncology Series,914
8,Nuclear Med Series,423
9,OB Series,5


In [219]:
%%bigquery
SELECT accomodation, COUNT(accomodation) as cnt
FROM (
    SELECT * FROM njr_labutil.lradt
    WHERE pat_lv_of_care = '' AND pat_class = 'Inpatient'
)
GROUP BY accomodation
ORDER BY cnt desc

,accomodation,cnt
0,Labor & Delivery,100983
1,PEDS MED/SURG Level 2,44914
2,Critical Care Level 3,38298
3,NICU Level 3 (174),30878
4,OB Level 4,27396
5,PEDS MED/SURG Level 3,23917
6,Oncology Level 3,22244
7,,22062
8,Oncology Level 2,21984
9,ECMO,20998


In [232]:
%%bigquery
SELECT * FROM njr_labutil.lradt
WHERE
    (pat_class = 'Inpatient' OR pat_class = 'Surgery Admit')
    AND pat_service NOT IN (
        'Obstetrics', 'Maternal-Fetal Medicine',
        'Reproductive Endocrinology and Infertility',
        'Gynecology & Obstetrics', 'Gynecology',
        'Intensive Care', 'Neonatology', 'Cardiovascular Intensive Care')
    AND (pat_lv_of_care = 'Acute Care'
        OR (pat_lv_of_care='' AND accomodation IN (
            'PEDS MED/SURG Level 1','PEDS MED/SURG Level 2', 'PEDS MED/SURG Level 3',
            'PEDS Med/Surg',
            'CCP Med/Surg Level 1','CCP Med/Surg Level 2','CCP Med/Surg Level 3',
            'PEDS TXP Med/Surg Level 1', 'PEDS TXP Med/Surg Level 2','PEDS TXP Med/Surg Level 3',
            'Oncology Level 1', 'Oncology Level 2', 'Oncology Level 3', 'Oncology Level 4'
            )
        )
    )
LIMIT 10

,uid,anon_id,pat_enc_csn_id_coded,order_id_coded,component_id,order_type,proc_code,group_lab_name,lab_name,base_name,...,department_id,pat_service,pat_lv_of_care,accomodation,from_base_class,to_base_class,in_event_type,out_event_type,event_id_coded,event_type
0,1463857,JC586854,309567035,702849975,797,Lab,LAB15,"Metabolic Panel, Basic","Urea Nitrogen, Ser/Plas",BUN,...,10201101,Stem Cell Transplant,Acute Care,SCT,Inpatient,Inpatient,Census,Census,352024,Census
1,1781918,JC586854,309567035,703683058,2507,Lab,LAB347,Urinalysis Macroscopic Only,Blood,UBLOOD,...,10201101,Stem Cell Transplant,Acute Care,SCT,Inpatient,Inpatient,Census,Census,438241,Census
2,1781334,JC586854,309567035,703683066,483,Lab,LAB293,CBC with Diff,RDW,RDW,...,10201101,Stem Cell Transplant,Acute Care,SCT,Inpatient,Inpatient,Census,Census,438241,Census
3,1761220,JC586854,309567035,703644511,6497,Lab,LAB347,Urinalysis Macroscopic Only,pH,UPH,...,10201101,Stem Cell Transplant,Acute Care,SCT,Inpatient,Inpatient,Census,Census,433437,Census
4,1760727,JC586854,309567035,703644516,6046,Lab,LAB293,CBC with Diff,MCV,MCV,...,10201101,Stem Cell Transplant,Acute Care,SCT,Inpatient,Inpatient,Census,Census,433437,Census
5,1811104,JC586854,309567035,703781705,6632,Lab,LAB17,"Metabolic Panel, Comprehensive","Protein, Total",TP,...,10201101,Stem Cell Transplant,Acute Care,SCT,Inpatient,Inpatient,Census,Census,446929,Census
6,2218369,JC586854,309567035,704897885,4668,Lab,LAB52,"Bilirubin Direct, Ser/Plas","Bilirubin, Direct",DBIL,...,10201101,Cardiac Cath,Acute Care,SCT,Inpatient,Inpatient,Transfer In,,553192,Transfer In
7,1587304,JC586854,309567035,703212235,492,Lab,LAB293,CBC with Diff,"EOS, %",EOS,...,10201101,Stem Cell Transplant,Acute Care,SCT,Inpatient,Inpatient,Census,Census,386111,Census
8,1781703,JC586854,309567035,703683066,499,Lab,LAB293,CBC with Diff,"EOS, ABS",EOSAB,...,10201101,Stem Cell Transplant,Acute Care,SCT,Inpatient,Inpatient,Census,Census,438241,Census
9,1353054,JC586854,309567035,702577412,6046,Lab,LAB293,CBC with Diff,MCV,MCV,...,10201101,Stem Cell Transplant,Acute Care,SCT,Inpatient,Inpatient,Transfer In,,327135,Transfer In
